In [1]:
from langchain.chains import SimpleSequentialChain

In [1]:
import os
import eunomia
import pickle


C:\Users\mehra\anaconda3\envs\agent\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
paper_id = '1'
docs_processor = eunomia.LoadDoc('../Data/SI_PDF/', paper_id=paper_id)
sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments', 'references\n'],
                                             chunk_size=1000, chunk_overlap=10, chunking_type='fixed-size')

In [3]:
from langchain.chains import RetrievalQA

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

k = 5
# Embedding_model = 'text-davinci-003' # text-embedding-ada-002
Embedding_model = 'text-embedding-ada-002' # text-embedding-ada-002


faiss_index_path = f"../Data/chosen_papers/faiss/{Embedding_model}/faiss_index_{Embedding_model}_{paper_id}.pkl"
just_load = True
if os.path.isfile(faiss_index_path) and just_load:
    print('loadded Embeddings')
    with open(faiss_index_path, "rb") as f:
        faiss_index = pickle.load(f)
else:
    with open(faiss_index_path, "wb") as f:  # text-embedding-ada-002
        faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))    # 'text-davinci-003
        pickle.dump(faiss_index, f)

# faiss_index = LanceDB.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))

retriever  = faiss_index.as_retriever(search_type="mmr", search_kwargs={"k":k, 'fetch_k': 50}) # Maximum Marginal Relevance Retrieval


loadded Embeddings


In [4]:
from langchain.llms import OpenAI
from langchain.agents import Tool, tool
from langchain.chat_models  import ChatOpenAI
from langchain.chains import RetrievalQA
llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-16k') # gpt-3.5-turbo-16k
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=False,)

C:\Users\mehra\anaconda3\envs\agent\lib\site-packages\langchain\llms\openai.py:201: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\mehra\anaconda3\envs\agent\lib\site-packages\langchain\llms\openai.py:786: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [5]:
Rule = f"""
There are only 3 options for water stability: \
1. Stable \
2. Unstable \
3. Not provided\
For 1. Stable, \
No change after exposure, soaking or boiling in water or an aqueous solution.\
Water adsorption isotherm should have a steep uptake.\
For 2. Unstable, \
The MOF will decompose after exposure \
to humid environment or partially dissolves in water. \
For 3. Not provided, \
If you don't know or cannot justfy 1 or 2
"""

# Steep uptake behavior is from this paper: https://pubs.acs.org/doi/epdf/10.1021/ja306812r

In [6]:
prompt3a = f"""
    You are an expert chemist. The doc tool describes the water stability properties of a few materials including \
    Metal-Organic Frameworks(MOF) or coordination polymers, Coordination Networks or ZIF compounds. Please find \
    the full name for materials, and their \
    stability in water. \
    Your final answer should contain the following:\
    Your first output is the water stability of each MOF. \
    Your second output for each mof should be a probability score ranging between [0, 1].\
    This probability score shows how certain you are in your answer.\
    Your third output should be exact sentences without any changes from the document that justifies your decision.\
    This should be "Not provided" if you cannot find water stability.\
    Rule: {Rule}\
    If there are multiple MOFs, please list them all and put "MOF name" before them. \
    You should only respond in JSON format as described below. 
    Make sure it is parsable using Python json.load.
    Make sure "MOF name"s do not have duplicates
    If information is not provided, state it as not provided.\
    """

In [7]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
qa.run(prompt3a)

'{\n  "MOF-11": {\n    "water_stability": "Unstable",\n    "probability_score": 1,\n    "justification": "Bio-MOF-11 dissolved rapidly in water, as evidenced by PXRD (Fig. 10); SEM images could not be obtained."\n  },\n  "MOF-12": {\n    "water_stability": "Unstable",\n    "probability_score": 1,\n    "justification": "Bio-MOF-12 dissolved partially in water. The intensity of their signature diﬀraction lines decreased substantially (Fig. 10); SEM images of the soaked samples revealed significant crystallite fragmentation and pitting of the crystal surfaces (Fig. S21 †)."\n  },\n  "MOF-13": {\n    "water_stability": "Unstable",\n    "probability_score": 1,\n    "justification": "Bio-MOF-13 dissolved partially in water. The intensity of their signature diﬀraction lines decreased substantially (Fig. 10); SEM images of the soaked samples revealed significant crystallite fragmentation and pitting of the crystal surfaces (Fig. S21 †)."\n  },\n  "MOF-14": {\n    "water_stability": "Stable",\n

In [9]:
@tool
def read_context(input):
    '''
    use this as the primary source of context information when you are asked the input.
    Always search for the answers using this tool first, don't make up answers yourself
    '''
    llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-16k') # gpt-3.5-turbo-16k
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm, chain_type="stuff", retriever=retriever)
    return qa_chain({"query": input})



In [18]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.callbacks import get_openai_callback

read_context_tool = create_retriever_tool(
    retriever, 
    "Read_paper",
    "Use this as the primary source of context information when you are asked the question about MOFs. Always search for the answers using this tool first, don't make up answers yourself"
)

tools = [
    # Tool(
    #     name = "Read_paper",
    #     func=qa.run,
    #     description=f"Useful for when you need to answer questions about MOFs."
    # )]
    read_context_tool]
    #eval_justification]


In [19]:
# from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
# agent_executor = create_conversational_retrieval_agent(llm, tools, verbose=True)

In [25]:

# with get_openai_callback() as cb:
#     agent_executor(prompt3a)
#     print(cb)

In [117]:
tools = [
    Tool(
        name = "doc",
        func=qa.run,
        description=f"Always use this tool for context. Use the original prompt for this tool to get context about water stability of MOFs."
    )]

In [118]:
# qa.run(prompt3a)

In [119]:
from llama_index import GPTListIndex, GPTIndexMemory
from langchain.agents import initialize_agent
from langchain.agents import AgentType
# index = GPTListIndex([])
# memory = GPTIndexMemory(index=GPTListIndex([]), memory_key="chat_history", query_kwargs={"response_mode": "compact"})
agent_llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-16k')
agent_chain = initialize_agent(tools, agent_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True, memory=None, handle_parsing_errors=True,
                              prefix= prompt3a)

In [120]:
with get_openai_callback() as cb:
    result = agent_chain.run(prompt3a)
    print(cb)



> Entering new AgentExecutor chain...
I need to use the doc tool to find the water stability properties of MOFs.
Action: doc
Action Input: water stability of MOFs
Observation: Based on the information provided, the water stability of the bio-MOFs was evaluated by soaking them in water for different periods of time and then evaluating their stability using PXRD and SEM imaging. Bio-MOF-11 dissolved rapidly in water, while bio-MOFs-12 and 13 dissolved partially. Bio-MOF-13, with butyrate, was noticeably more stable in water than bio-MOF-12, with propionate. Bio-MOF-14, with valerate groups, was the most stable and showed no loss of crystallinity or significant crystal degradation after soaking in water for 1 hour, 7 days, or 30 days. Additionally, CO2 isotherms collected for bio-MOF-14 soaked in water for 7 or 30 days showed no loss in crystallinity and closely matched the isotherm of the non-water-treated sample, indicating that water did not affect the porosity of the material.
Thoug

In [121]:
result

'{\n  "MOF name": "bio-MOF-11",\n  "Water Stability": "Unstable",\n  "Probability Score": 1,\n  "Justification": "Bio-MOF-11 dissolved rapidly in water."\n},\n{\n  "MOF name": "bio-MOF-12",\n  "Water Stability": "Unstable",\n  "Probability Score": 1,\n  "Justification": "Bio-MOF-12 dissolved partially in water."\n},\n{\n  "MOF name": "bio-MOF-13",\n  "Water Stability": "Unstable",\n  "Probability Score": 1,\n  "Justification": "Bio-MOF-13 dissolved partially in water."\n},\n{\n  "MOF name": "bio-MOF-14",\n  "Water Stability": "Stable",\n  "Probability Score": 1,\n  "Justification": "Bio-MOF-14 showed no loss of crystallinity or significant crystal degradation after soaking in water for 1 hour, 7 days, or 30 days."\n}'

In [173]:


from langchain.agents import AgentOutputParser
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException

class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        '''
        Use this tool last to parse the final answer to a dictionary.
        '''
        if "Final Answer:" in llm_output:
            print('FUCK')
            from typing import List
            from pydantic import BaseModel, Field, validator
            from langchain.output_parsers import OutputFixingParser, PydanticOutputParser
    
            class MOF(BaseModel):
                name: str = Field(description="name of a MOF")
                stability: str = Field(description="choose only one: stable or unstable or not provided")
                score: float = Field(description="probability score of prediction")
                justification: str = Field(description="justification for prediction")
        
            class MOFList(BaseModel):
                MOFs: List[MOF]
        
            parser = PydanticOutputParser(pydantic_object=MOFList)
            llm_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(temperature=0))
        
            try:
                parsed_result = llm_parser.parse(llm_output)
            except ValueError as e:
                raise ValueError(f"Failed to parse MOF: {e}")
        
            dict_response = {mof.name: {"Predicted Stability": mof.stability, "Score": mof.score, "Justification": mof.justification} for mof in parsed_result.MOFs}
            new_dict = {key: {**value} for key, value in dict_response.items()}
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": new_dict},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()
    # def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
    #     # Check if agent should finish
    #     if "Final Answer:" in llm_output:
    #         return AgentFinish(
    #             # Return values is generally always a dictionary with a single `output` key
    #             # It is not recommended to try anything else at the moment :)
    #             return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
    #             log=llm_output,
    #         )
    #     # Parse out the action and action input
    #     regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
    #     match = re.search(regex, llm_output, re.DOTALL)
    #     if not match:
    #         raise OutputParserException(f"Could not parse LLM output: `{llm_output}`")
    #     action = match.group(1).strip()
    #     action_input = match.group(2)
    #     # Return the action and action input
    #     return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)



In [181]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

mof_name_schema = ResponseSchema(name="MOF_name",
                             description="This is the name of the MOF")

water_stability_schema = ResponseSchema(name="water_stability",
                                      description="This is the stability of the MOF: Stable, Unstable or Not provided")

probability_score_schema = ResponseSchema(name="probability_score",
                                      description="This is an integer score between 1-10")

justification_schema = ResponseSchema(name="justification",
                                    description="This is sentense that justifies the predicted water stability.")

response_schemas = [mof_name_schema, 
                    water_stability_schema,
                    probability_score_schema,
                    justification_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [179]:
import langchain
langchain.debug = False

In [198]:
prompt3a = f"""
    You are an expert chemist. The doc tool describes the water stability properties of a few materials including \
    Metal-Organic Frameworks(MOF) or coordination polymers, Coordination Networks or ZIF compounds. Please find \
    the full name for materials, and their \
    stability in water. \
    Your final answer should contain the following:\
    Your first output is the water stability of each MOF. \
    Your second output for each mof should be a probability score ranging between [0, 1].\
    This probability score shows how certain you are in your answer.\
    Your third output should be exact sentences without any changes from the document that justifies your decision.\
    This should be "Not provided" if you cannot find water stability.\
    Rule: {Rule}\
    If there are multiple MOFs, please list them all and put "MOF name" before them. \
    You should only respond in JSON format as described below. 
    Make sure it is parsable using Python json.load.
    Make sure "MOF name"s do not have duplicates
    If information is not provided, state it as not provided.\
    """
    
#     """

Rule = f"""
There are only 3 options for water stability: \
1. Stable \
2. Unstable \
3. Not provided\
For 1. Stable, \
No change after exposure, soaking or boiling in water or an aqueous solution.\
Water adsorption isotherm should have a steep uptake.\
For 2. Unstable, \
The MOF will decompose after exposure \
to humid environment or partially dissolves in water. \
For 3. Not provided, \
If you don't know or cannot justfy 1 or 2
"""

@tool
def doc_retrival(input):
    """Use this as the primary source of context information when you are asked the question about MOFs.
    Always search for the answers using this tool first, don't make up answers yourself"""
    paper_id = '1'
    docs_processor = eunomia.LoadDoc('../Data/SI_PDF/', paper_id=paper_id)
    sliced_pages = docs_processor.process(['references ', 'acknowledgement', 'acknowledgments', 'references\n'],
                                                 chunk_size=1000, chunk_overlap=10, chunking_type='fixed-size')

    from langchain.embeddings.openai import OpenAIEmbeddings
    from langchain.vectorstores import FAISS

    k = 5
    # Embedding_model = 'text-davinci-003' # text-embedding-ada-002
    Embedding_model = 'text-embedding-ada-002' # text-embedding-ada-002
    
    
    faiss_index_path = f"../Data/chosen_papers/faiss/{Embedding_model}/faiss_index_{Embedding_model}_{paper_id}.pkl"
    just_load = False
    if os.path.isfile(faiss_index_path) and just_load:
        print('loadded Embeddings')
        with open(faiss_index_path, "rb") as f:
            faiss_index = pickle.load(f)
    else:
        with open(faiss_index_path, "wb") as f:  # text-embedding-ada-002
            faiss_index = FAISS.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))    # 'text-davinci-003
            pickle.dump(faiss_index, f)

    # faiss_index = LanceDB.from_documents(sliced_pages, OpenAIEmbeddings(model=Embedding_model))
    
    retriever  = faiss_index.as_retriever(search_type="mmr", search_kwargs={"k":k, 'fetch_k': 50}) # Maximum Marginal Relevance Retrieval
        
    from langchain.llms import OpenAI
    from langchain.chains import RetrievalQAWithSourcesChain
    llm = OpenAI(temperature=0, model_name='gpt-3.5-turbo-16k') # gpt-3.5-turbo-16k
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    result = qa({"query": prompt3a})
    return result['result']

# Steep uptake behavior is from this paper: https://pubs.acs.org/doi/epdf/10.1021/ja306812r

prefix = """You are an expert chemist. The doc tool describes the water stability properties of a few materials including \
    Metal-Organic Frameworks(MOF) or coordination polymers, Coordination Networks or ZIF compounds.
    """


output_format = f"""
    Your final answer should be in JSON format contain the following keys:\
    Your first key is the water stability of each MOF. \
    Your second key for each mof should be a probability score ranging between [0, 1].\
    This probability score shows how certain you are in your answer.\
    Your third key should be exact sentences without any changes from the document that justifies your decision.\
    This should be "Not provided" if you cannot find water stability.\
    If there are multiple MOFs, please list them all and put "MOF name" before them. \
    Make sure it is parsable using Python json.load.
    Make sure "MOF name"s do not have duplicates
    If information is not provided, state it as not provided.\
        """

@tool
def eval_justification(justification):
    '''
    Always use this tool at the last step. This function checks if the justification talks about the water stability. 
    '''
    import openai
    model="gpt-3.5-turbo"
    prompt = f"""Does the below sentence really describe information on water stability on all found MOFS?\
            If not remove find a better justification.
            {justification}
            """
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]

agent_llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-16k')
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history")

tools = [
    Tool(
        name = "doc",
        func=qa.run,
        description=f"Always use this tool for context. Use the original prompt for this tool to get context about water stability of MOFs."
    ), eval_justification]

# tools = [doc_retrival]
agent_chain = initialize_agent(tools, agent_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                               verbose=True, memory=memory, handle_parsing_errors=True,
                              prefix= None, suffix=None, output_parser=None)



with get_openai_callback() as cb:
    result = agent_chain.run(input=prompt3a)
    print(cb)



> Entering new AgentExecutor chain...
I need to use the doc tool to find information about the water stability of MOFs. I should also keep track of the sentences that justify my decision.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I need to use the doc tool to find information about the water stability of MOFs. I should also keep track of the sentences that justify my decision.
Action: doc
Action Input: water stability of MOFs
Observation: Based on the information provided, the water stability of the bio-MOFs was evaluated by soaking them in water for different periods of time and then evaluating their stability using PXRD and SEM imaging. Bio-MOF-11 dissolved rapidly in water, while bio-MOFs-12 and 13 dissolved partially. Bio-MOF-13, with butyrate, was noticeably more stable in water than bio-MOF-12, with propionate. Bio-MOF-14, with valerate groups, was the most stable and showed no loss of crystallinity or significant crystal degradation after soaking i

In [205]:
def parse_to_dict(result, paper_id):
    from typing import List
    from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
    from pydantic import BaseModel, Field, validator

    class MOF(BaseModel):
        name: str = Field(description="name of a MOF")
        stability: str = Field(description="choose only one: stable or unstable or not provided")
        score: float = Field(description="probability score of prediction")
        justification: str = Field(description="justification for prediction")

    class MOFList(BaseModel):
        MOFs: List[MOF]

    parser = PydanticOutputParser(pydantic_object=MOFList)
    llm_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI(temperature=0))

    try:
        parsed_result = llm_parser.parse(result)
    except ValueError as e:
        raise ValueError(f"Failed to parse MOF: {e}")

    dict_response = {mof.name: {"Predicted Stability": mof.stability, "Score": mof.score, "Justification": mof.justification} for mof in parsed_result.MOFs}
    new_dict = {key: {**value, 'Paper id': paper_id} for key, value in dict_response.items()}
    return new_dict


parse_to_dict(result, paper_id)

{'bio-MOF-11': {'Predicted Stability': 'Unstable',
  'Score': 1.0,
  'Justification': 'Based on the information provided, the water stability of the bio-MOFs was evaluated by soaking them in water for different periods of time and then evaluating their stability using PXRD and SEM imaging. Bio-MOF-11 dissolved rapidly in water.',
  'Paper id': '1'},
 'bio-MOF-12': {'Predicted Stability': 'Unstable',
  'Score': 1.0,
  'Justification': 'Based on the information provided, the water stability of the bio-MOFs was evaluated by soaking them in water for different periods of time and then evaluating their stability using PXRD and SEM imaging. Bio-MOFs-12 dissolved partially in water.',
  'Paper id': '1'},
 'bio-MOF-13': {'Predicted Stability': 'Unstable',
  'Score': 1.0,
  'Justification': 'Based on the information provided, the water stability of the bio-MOFs was evaluated by soaking them in water for different periods of time and then evaluating their stability using PXRD and SEM imaging. Bi